In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('../../raw_data/State_Population_Data_2000_2019.xls', sheet_name="Sheet1")

In [3]:
data.head()

,observation_date,state,population
0,1950-01-01,ak,135.0
1,1950-01-01,al,3058.0
2,1950-01-01,ar,1908.0
3,1950-01-01,az,756.0
4,1950-01-01,ca,10677.0


In [4]:
data['day'] = data['observation_date'].apply(lambda x: x.strftime("%Y-%m-%d"))
data['month'] = data['observation_date'].apply(lambda x: x.strftime("%Y-%m"))
data['month_number'] = data['observation_date'].apply(lambda x: int(x.month))
data['year_number'] = data['observation_date'].apply(lambda x: int(x.year))

In [5]:
seasons = ['winter','winter','spring','spring','spring','summer','summer','summer','autumn','autumn','autumn','winter']
month_to_season = dict(zip(range(1,13), seasons))
data['season'] = data.month_number.map(month_to_season)
data['year_season'] = data['year_number'].astype(str)+'-'+data['season']

In [6]:
def value_count_func (series_):
    return len(series_.value_counts())

def state_season_df (data_, state_):
    
    data_state = data_[data_['state'] == state_]
    
    data_state_season = data_state.groupby(by='year_season').agg({'population':'first'})
       
    data_state_season['state'] = state_
    
    return data_state_season.reset_index().set_index('state')

In [7]:
STATES_ = list(data.state.unique()) 
x_ = pd.DataFrame()

for st_ in STATES_:
    
    y_ = state_season_df (data, st_)
    x_ = pd.concat([x_,y_])
    
x_ = x_.reset_index()

In [8]:
x_[['year', 'season']] = x_['year_season'].str.split('-',expand=True)
x_ = x_[x_['year']>'1974'].reset_index(drop=True)
y_ = x_.iloc[np.arange(len(x_)).repeat(4)].reset_index(drop=True)
y_.drop(columns = ['season', 'year_season'], inplace = True)
seasons_ = pd.DataFrame({'season':['winter', 'spring','summer', 'autumn']})
seasons_ = seasons_.iloc[np.tile(np.arange(len(seasons_)), len(x_))]. reset_index(drop = True)
merged = y_.join(seasons_, lsuffix='_caller', rsuffix='_other')
merged['year_season'] = merged['year'].astype(str)+'-'+merged['season']


In [9]:
merged.head()

,state,population,year,season,year_season
0,ak,376.170,1975,winter,1975-winter
1,ak,376.170,1975,spring,1975-spring
2,ak,376.170,1975,summer,1975-summer
3,ak,376.170,1975,autumn,1975-autumn
4,ak,400.969,1976,winter,1976-winter


In [10]:
merged.to_csv('../../raw_data/population_by_state_season.csv',header=True,index=False)